# 📘 Data Preprocessing Overview
#### Cyanobacteria Bloom Prediction in U.S. Reservoirs (1987–2018)

### 1️⃣ Data Sources Used

We integrated multiple long-term environmental datasets collected from **20 U.S. reservoirs** between **1987 and 2018**:

* **Cyanobacteria observations**
  Monthly cyanobacterial density (cells/ml)

* **Water temperature**

  * Deep-water temperature
  * Surface-water temperature

* **Water quality**

  * Deep dissolved oxygen (DO)

* **Nutrient variables**

  * Phosphorus and nitrogen species (TP, TKN, NH₃, NOₓ, etc.)
  * Inflow nutrient concentrations
  * Chlorophyll-a and Secchi depth

* **Watershed land-cover data**

  * Forest, agriculture, urban, wetlands, and other land-use classes (NLCD 2011)

---

### 2️⃣ Temporal Aggregation

* All datasets were **aligned to a monthly time scale**
* Daily or irregular measurements were **aggregated using monthly means**
* Each observation corresponds to:

  ```
  Reservoir – Year – Month
  ```

This ensured **temporal consistency** across all data sources.

---

### 3️⃣ Cyanobacteria Data Processing

* Raw cyanobacteria measurements were converted to **monthly mean density**
* Data were grouped by:

  * Reservoir
  * Year
  * Month
* This variable serves as the **target variable** for prediction

---

### 4️⃣ Environmental Variable Processing

#### 🔹 Deep & Surface Temperature

* Temperature datasets were originally stored in **wide format**
* Converted to **long format** and parsed correctly as dates
* Extracted `year` and `month`
* Aggregated to monthly means per reservoir

#### 🔹 Deep Dissolved Oxygen

* Similar restructuring from wide to long format
* Monthly averaging applied
* Ensured alignment with temperature and cyanobacteria data

---

### 5️⃣ Nutrient Data Integration

* Nutrient concentrations were **static per reservoir**
* Converted all values to numeric format
* Merged directly using the reservoir identifier

---

### 6️⃣ Land-Cover Data Integration

* Watershed land-cover percentages from **NLCD 2011**
* Represent static catchment characteristics:

  * Forest cover
  * Agricultural land
  * Urban development
  * Wetlands
* Merged at the reservoir level
* Land-cover values are repeated across months for each reservoir

---

### 7️⃣ Data Cleaning & Standardization

* Reservoir names standardized (uppercase, trimmed)
* Invalid dates and records removed
* Missing numeric values filled using **median imputation**
* Column names cleaned for modeling compatibility

---

### 8️⃣ Final Dataset Structure

The final preprocessed dataset contains:

* **742 observations**
* **48 predictor variables**
* **0 missing values**
* Fully numeric and ML-ready format

Each row represents:

```
One reservoir × One month × Environmental conditions → Cyanobacteria density
```

---

### 9️⃣ Why This Preprocessing Matters

This preprocessing pipeline ensures:

* ✅ No temporal mismatch between datasets
* ✅ No data leakage
* ✅ Physically and ecologically consistent variables
* ✅ Fair comparison across different ML and DL models

As a result, any predictive model trained on this dataset will reflect **true environmental drivers** of cyanobacteria blooms.

---

### 10️⃣ Ready for Modeling

After preprocessing, the dataset is suitable for:

* Traditional ML models (RF, XGBoost, SVR)
* Hybrid deep learning models (CNN-LSTM)
* Transformer-based time-series models
* Graph Neural Networks (GNNs)
* Reservoir-wise performance evaluation (R², RMSE, MAE)

---

#### ✅ In one sentence:

> *All heterogeneous environmental datasets were cleaned, temporally aligned, aggregated, and merged into a single monthly, reservoir-wise dataset suitable for robust cyanobacteria bloom prediction.*


In [ ]:
## Cell 1 – Import Required Libraries

import pandas as pd
import numpy as np

In [ ]:
## Cell 2 – Load Raw Data

CYANO_PATH     = "/kaggle/input/cyanobacterial-data/Cynobacterial_Data/Cyanobacteria_data (1).xlsx"
DEEP_TEMP_PATH = "/kaggle/input/cyanobacterial-data/Cynobacterial_Data/Deep_Temperatures_standardized_FINAL.xlsx"
SURF_TEMP_PATH = "/kaggle/input/cyanobacterial-data/Cynobacterial_Data/Surface_Temperatures_standardized_FINAL.xlsx"
DEEP_DO_PATH   = "/kaggle/input/cyanobacterial-data/Cynobacterial_Data/Deep_DO_standardized_FINAL.xlsx"
NUT_PATH       = "/kaggle/input/cyanobacterial-data/Cynobacterial_Data/Means_of_reservoir_nutrients.xlsx"
LAND_PATH      = "/kaggle/input/cyanobacterial-data/Cynobacterial_Data/NLCD_watershed_land_cover.xlsx"

cyano_df     = pd.read_excel(CYANO_PATH, sheet_name="All_data")
deep_temp_df = pd.read_excel(DEEP_TEMP_PATH, sheet_name="All_reservoirs")
surf_temp_df = pd.read_excel(SURF_TEMP_PATH, sheet_name="All_reservoirs")
deep_do_df   = pd.read_excel(DEEP_DO_PATH, sheet_name="All_reservoirs")
nut_df       = pd.read_excel(NUT_PATH, sheet_name="Means_of_variables")
land_df      = pd.read_excel(LAND_PATH, sheet_name="NLCD_2011")


In [ ]:
##  Cell 3 – CELL 3 — Cyanobacteria Monthly Aggregation

cyano_df['date'] = pd.to_datetime(cyano_df['date'], errors='coerce')

cyano_agg = (
    cyano_df
    .groupby([
        'reservoir',
        cyano_df['date'].dt.year.rename('year'),
        cyano_df['date'].dt.month.rename('month')
    ])['density_cells/ml']
    .mean()
    .reset_index()
)

cyano_agg['reservoir'] = cyano_agg['reservoir'].str.strip().str.upper()
cyano_agg.head()


In [ ]:
## CELL 4 — Initialize Master DataFrame

df = cyano_agg.copy()
df.head()


In [ ]:
## CELL 5 — Process Deep Temperature

deep_blocks = []

for col in deep_temp_df.columns:
    if col.startswith('reservoir'):
        suffix = col.replace('reservoir', '')
        try:
            sub = deep_temp_df[
                [
                    f'reservoir{suffix}',
                    f'date{suffix}',
                    f'year{suffix}',
                    f'month{suffix}',
                    f'Temp{suffix}',
                    f'slope{suffix}',
                    f'Predicted{suffix}',
                    f'Residual{suffix}',
                    f'15.5th{suffix}'
                ]
            ].copy()

            sub.columns = [
                'reservoir','raw_date','year','month',
                'deep_Temp','deep_slope','deep_Predicted',
                'deep_Residual','deep_15_5th'
            ]

            deep_blocks.append(sub)
        except KeyError:
            pass

deep_long = pd.concat(deep_blocks, ignore_index=True)

deep_long['raw_date'] = deep_long['raw_date'].astype(str).str.split('.').str[0]
deep_long['date'] = pd.to_datetime(deep_long['raw_date'], format='%Y%m%d', errors='coerce')

deep_long = deep_long.dropna(subset=['reservoir','date','deep_Temp'])

deep_temp_monthly = (
    deep_long
    .groupby(['reservoir','year','month'])
    .mean(numeric_only=True)
    .reset_index()
)

deep_temp_monthly = deep_temp_monthly.query("year >= 1987 and year <= 2018")
deep_temp_monthly['reservoir'] = deep_temp_monthly['reservoir'].str.strip().str.upper()


In [ ]:
## CELL 6 — Merge Deep Temperature

df = df.merge(
    deep_temp_monthly,
    on=['reservoir','year','month'],
    how='left'
)

In [ ]:
## CELL 7 — Process Deep DO

do_blocks = []

do_cols = [c for c in deep_do_df.columns if 'do' in c.lower() or 'oxygen' in c.lower()]

for col in deep_do_df.columns:
    if col.startswith('reservoir'):
        suffix = col.replace('reservoir','')
        try:
            do_col = next(c for c in do_cols if c.endswith(suffix))

            sub = deep_do_df[
                [
                    f'reservoir{suffix}',
                    f'date{suffix}',
                    f'year{suffix}',
                    f'month{suffix}',
                    do_col
                ]
            ].copy()

            sub.columns = ['reservoir','raw_date','year','month','deep_DO']
            do_blocks.append(sub)

        except (KeyError, StopIteration):
            pass

deep_do_long = pd.concat(do_blocks, ignore_index=True)

deep_do_long['raw_date'] = deep_do_long['raw_date'].astype(str).str.split('.').str[0]
deep_do_long['date'] = pd.to_datetime(deep_do_long['raw_date'], format='%Y%m%d', errors='coerce')

deep_do_long = deep_do_long.dropna(subset=['reservoir','date','deep_DO'])

deep_do_monthly = (
    deep_do_long
    .groupby(['reservoir','year','month'])
    .mean(numeric_only=True)
    .reset_index()
)

deep_do_monthly = deep_do_monthly.query("year >= 1987 and year <= 2018")
deep_do_monthly['reservoir'] = deep_do_monthly['reservoir'].str.strip().str.upper()


In [ ]:
## CELL 8 — Merge Deep DO

df = df.merge(
    deep_do_monthly,
    on=['reservoir','year','month'],
    how='left'
)


In [ ]:
## CELL 9 — Build Surface Temperature (Long Format)

surf_blocks = []

for col in surf_temp_df.columns:
    if col.startswith('Reservoir'):
        suffix = col.replace('Reservoir','')
        try:
            sub = surf_temp_df[
                [
                    f'Reservoir{suffix}',
                    f'Date{suffix}',
                    f'temperature{suffix}',
                    f'Predicted{suffix}',
                    f'Residual{suffix}',
                    f'15.5th{suffix}'
                ]
            ].copy()

            sub.columns = [
                'reservoir','raw_date',
                'surface_Temp','surface_Predicted',
                'surface_Residual','surface_15_5th'
            ]

            surf_blocks.append(sub)
        except KeyError:
            pass

surf_long = pd.concat(surf_blocks, ignore_index=True)

surf_long['raw_date'] = surf_long['raw_date'].astype(str).str.split('.').str[0]
surf_long['date'] = pd.to_datetime(surf_long['raw_date'], format='%Y%m%d', errors='coerce')

surf_long = surf_long.dropna(subset=['reservoir','date','surface_Temp'])

surf_long['year'] = surf_long['date'].dt.year
surf_long['month'] = surf_long['date'].dt.month
surf_long['reservoir'] = surf_long['reservoir'].str.strip().str.upper()


In [ ]:
## CELL 10 — Process & Merge Surface Temperature

group_keys = ['reservoir','year','month']

numeric_cols = [
    c for c in surf_long.select_dtypes(include=np.number).columns
    if c not in ['year','month']
]

surf_temp_monthly = (
    surf_long
    .groupby(group_keys)[numeric_cols]
    .mean()
    .reset_index()
)

surf_temp_monthly = surf_temp_monthly.query("year >= 1987 and year <= 2018")

df = df.merge(
    surf_temp_monthly,
    on=['reservoir','year','month'],
    how='left'
)


In [ ]:
## CELL 11 — Merge Nutrients

nut_df.columns = nut_df.columns.str.strip()
nut_df['reservoir'] = nut_df['reservoir'].str.strip().str.upper()

for col in nut_df.columns:
    if col != 'reservoir':
        nut_df[col] = pd.to_numeric(nut_df[col], errors='coerce')

df = df.merge(nut_df, on='reservoir', how='left')


In [ ]:
## CELL 12 — Merge Land Cover

land_df.columns = land_df.columns.str.strip()
land_df = land_df.rename(columns={'Reservoir':'reservoir'})
land_df['reservoir'] = land_df['reservoir'].str.strip().str.upper()

df = df.merge(land_df, on='reservoir', how='left')


In [ ]:
## CELL 13 — Final Cleaning

if 'date' in df.columns:
    df = df.drop(columns=['date'])

num_cols = df.select_dtypes(include=np.number).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())


In [ ]:
## CELL 14 — Final Sanity Check

df.info()
df.head()


In [ ]:
## CELL 15 — Rename columns with spaces & slashes (ML-friendly)

df = df.rename(columns={
    'density_cells/ml': 'density_cells_ml',
    'Barren Land': 'lc_barren',
    'Cultivated Crops': 'lc_crops',
    'Deciduous Forest': 'lc_deciduous_forest',
    'Developed High Intensity': 'lc_dev_high',
    'Developed Medium Intensity': 'lc_dev_medium',
    'Developed Low Intensity': 'lc_dev_low',
    'Developed Open Space': 'lc_dev_open',
    'Emergent Herbaceous Wetlands': 'lc_wetland_herb',
    'Evergreen Forest': 'lc_evergreen_forest',
    'Grassland/Herbaceous': 'lc_grassland',
    'Mixed Forest': 'lc_mixed_forest',
    'Open Water': 'lc_open_water',
    'Pasture/Hay': 'lc_pasture',
    'Shrub/Scrub': 'lc_shrub',
    'Woody Wetlands': 'lc_wetland_woody'
})


In [ ]:
## Save the final locked dataset

df.to_csv("cyanobacteria_preprocessed_master_FINAL.csv", index=False)


# 🔍 FEATURE IMPACT ANALYSIS
##### Cyanobacteria Bloom Drivers in U.S. Reservoirs (1987–2018)

In [ ]:
##  CELL 1 — Setup & Target / Feature Split
# Copy dataset to avoid accidental changes
data = df.copy()

# Rename target for convenience (if not already done)
data = data.rename(columns={'density_cells/ml': 'density_cells_ml'})

# Target variable
y = data['density_cells_ml']

# Drop non-feature columns
X = data.drop(columns=[
    'density_cells_ml',
    'reservoir',   # categorical, handled later
    'year'         # temporal info, optional
])


In [ ]:
## CELL 2 — Basic Feature Statistics (Sanity Check)

X.describe().T[['mean', 'std', 'min', 'max']].head(10)

### PART 1: CORRELATION ANALYSIS (GLOBAL) — Linear Relationships

In [ ]:
## CELL 3 — Correlation with Target Variable

corr_with_target = (
    data.corr(numeric_only=True)['density_cells_ml']
    .sort_values(ascending=False)
)

corr_with_target


In [ ]:
## CELL 4 — Top Positive & Negative Correlations
# Top drivers
top_positive = corr_with_target[1:11]
top_negative = corr_with_target[-10:]

top_positive, top_negative


### PART 2: RANDOM FOREST FEATURE IMPORTANCE — Nonlinear & Interaction Effects

In [ ]:
## CELL 5 — Train Random Forest Regressor

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=500,
    random_state=42,
    n_jobs=-1
)

rf.fit(X, y)


In [ ]:
## CELL 6 — Random Forest Feature Importance

rf_importance = (
    pd.Series(rf.feature_importances_, index=X.columns)
    .sort_values(ascending=False)
)

rf_importance.head(15)


In [ ]:
## CELL 7 — Plot Random Forest Importance

import matplotlib.pyplot as plt

rf_importance.head(15).plot(kind='barh', figsize=(8,6))
plt.gca().invert_yaxis()
plt.title("Random Forest Feature Importance (Global)")
plt.xlabel("Importance Score")
plt.show()


### PART 3: XGBOOST FEATURE IMPORTANCE - High-Resolution Nutrient Sensitivity

## CELL 8 — Train XGBoost Regressor

from xgboost import XGBRegressor

xgb = XGBRegressor(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb.fit(X, y)


In [ ]:
## CELL 9 — XGBoost Feature Importance

xgb_importance = (
    pd.Series(xgb.feature_importances_, index=X.columns)
    .sort_values(ascending=False)
)

xgb_importance.head(15)


In [ ]:
## CELL 10 — Plot XGBoost Importance

xgb_importance.head(15).plot(kind='barh', figsize=(8,6))
plt.gca().invert_yaxis()
plt.title("XGBoost Feature Importance (Global)")
plt.xlabel("Importance Score")
plt.show()


### PART 4: CONSOLIDATED FEATURE RANKING — Robust Consensus Drivers

In [ ]:
## CELL 11 — Combined Importance Table

importance_df = pd.DataFrame({
    'RF_importance': rf_importance,
    'XGB_importance': xgb_importance
}).fillna(0)

importance_df['mean_importance'] = importance_df.mean(axis=1)
importance_df.sort_values('mean_importance', ascending=False).head(15)


# 🧭 Reservoir-Wise Feature Impact Analysis
Understanding Bloom Drivers Across 20 U.S. Reservoirs

In [ ]:
## CELL 1 — Imports & Feature Setup

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor


In [ ]:
## CELL 2 — Define Target & Feature Columns

TARGET = 'density_cells_ml'

EXCLUDE_COLS = [
    TARGET,
    'reservoir',
    'year'
]

FEATURE_COLS = [c for c in df.columns if c not in EXCLUDE_COLS]


### PART 1: RANDOM FOREST FEATURE IMPORTANCE (PER RESERVOIR)

In [ ]:
## CELL 3 — Compute Feature Importance per Reservoir

reservoir_importance = {}

for reservoir in df['reservoir'].unique():
    sub = df[df['reservoir'] == reservoir]

    # Skip reservoirs with too few samples
    if len(sub) < 20:
        continue

    X = sub[FEATURE_COLS]
    y = sub[TARGET]

    rf = RandomForestRegressor(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    )

    rf.fit(X, y)

    importance = pd.Series(
        rf.feature_importances_,
        index=FEATURE_COLS
    ).sort_values(ascending=False)

    reservoir_importance[reservoir] = importance


In [ ]:
## CELL 4 — Top Drivers per Reservoir (Table)

top_features_per_reservoir = pd.DataFrame({
    r: imp.head(5).index.tolist()
    for r, imp in reservoir_importance.items()
}).T

top_features_per_reservoir


### 🔹 PART 2: VISUALIZE FEATURE IMPORTANCE (RESERVOIR-WISE)

In [ ]:
## CELL 5 — Plot Top Features for One Reservoir

def plot_reservoir_importance(reservoir, top_n=10):
    imp = reservoir_importance[reservoir].head(top_n)

    plt.figure(figsize=(7,5))
    imp.sort_values().plot(kind='barh')
    plt.title(f"Top {top_n} Drivers — Reservoir {reservoir}")
    plt.xlabel("Feature Importance")
    plt.show()

# Example
plot_reservoir_importance('BVR')


### PART 3: RESERVOIR CLASSIFICATION BY DOMINANT DRIVER

In [ ]:
## CELL 7 — Categorize Drivers

driver_groups = {
    'Nutrient': [
        'TIN_inflow_ppm','NOx_inflow_ppm','TP_ppb','TKN_ppm',
        'NH3_ppm','TON_ppm'
    ],
    'Temperature': [
        'surface_Temp','deep_Temp','surface_Predicted','deep_Predicted',
        'deep_slope'
    ],
    'Oxygen': ['deep_DO'],
    'LandCover': [
        'Agricultural','Forest','Developed',
        'lc_pasture','lc_crops'
    ],
    'Seasonality': ['month']
}


In [ ]:
## CELL 8 — Dominant Driver per Reservoir

reservoir_driver_summary = []

for reservoir, imp in reservoir_importance.items():
    scores = {}

    for group, features in driver_groups.items():
        scores[group] = imp[imp.index.isin(features)].sum()

    dominant_driver = max(scores, key=scores.get)

    reservoir_driver_summary.append({
        'reservoir': reservoir,
        'dominant_driver': dominant_driver,
        'scores': scores
    })

driver_df = pd.DataFrame(reservoir_driver_summary)
driver_df[['reservoir','dominant_driver']]


In [ ]:
## CELL 9 — Plot Driver Distribution
driver_df['dominant_driver'].value_counts().plot(
    kind='bar',
    figsize=(6,4),
    title='Dominant Bloom Drivers Across Reservoirs'
)
plt.ylabel('Number of Reservoirs')
plt.show()


# 🧬 Species Dominance Analysis
Identifying Cyanobacteria Species Driving Blooms in U.S. Reservoirs

### PART 1: GLOBAL SPECIES DOMINANCE

In [ ]:
## CELL 1 — Load & Prepare Species Data

# Load raw cyanobacteria data (species-level)
cyano_species = pd.read_excel(
    CYANO_PATH,
    sheet_name="All_data"
)

# Standardize columns
cyano_species.columns = cyano_species.columns.str.strip()

# Rename for clarity
cyano_species = cyano_species.rename(columns={
    'density_cells/ml': 'density_cells_ml'
})

cyano_species.head()


In [ ]:
## CELL 2 — Global Species Abundance

species_global = (
    cyano_species
    .groupby('sci_name')['density_cells_ml']
    .mean()
    .sort_values(ascending=False)
)

species_global.head(10)


In [ ]:
## CELL 3 — Plot Top Dominant Species (Global)

species_global.head(10).plot(
    kind='bar',
    figsize=(8,4),
    title='Top 10 Dominant Cyanobacteria Species (Global)'
)
plt.ylabel('Mean Cell Density (cells/ml)')
plt.show()


### PART 2: RESERVOIR-WISE SPECIES DOMINANCE

## CELL 4 — Species Dominance per Reservoir

species_reservoir = (
    cyano_species
    .groupby(['reservoir','sci_name'])['density_cells_ml']
    .mean()
    .reset_index()
)

species_reservoir.head()


In [ ]:
## CELL 5 — Top Species per Reservoir

top_species_reservoir = (
    species_reservoir
    .sort_values(['reservoir','density_cells_ml'], ascending=False)
    .groupby('reservoir')
    .head(3)
)

top_species_reservoir


In [ ]:
## CELL 6 — Heatmap of Species × Reservoir

pivot_species = species_reservoir.pivot_table(
    index='sci_name',
    columns='reservoir',
    values='density_cells_ml',
    aggfunc='mean'
)

plt.figure(figsize=(35,60))
plt.imshow(
    np.log1p(pivot_species.fillna(0)),
    aspect='auto'
)
plt.colorbar(label='log(1 + Density)')
plt.yticks(range(len(pivot_species.index)), pivot_species.index)
plt.xticks(range(len(pivot_species.columns)), pivot_species.columns, rotation=90)
plt.title('Species Dominance Across Reservoirs')
plt.show()


### PART 3: BLOOM CONTRIBUTION BY TAXONOMIC GROUP

In [ ]:
## CELL 7 — Taxonomic Group Contribution

taxa_contribution = (
    cyano_species
    .groupby('FinalTaxaType')['density_cells_ml']
    .mean()
    .sort_values(ascending=False)
)

taxa_contribution


In [ ]:
## CELL 8 — Plot Taxonomic Contribution

taxa_contribution.plot(
    kind='bar',
    figsize=(6,4),
    title='Bloom Contribution by Cyanobacteria Taxonomic Group'
)
plt.ylabel('Mean Cell Density')
plt.show()


### PART 4: SPECIES CONTRIBUTION TO BLOOM EVENTS

In [ ]:
## CELL 9 — High-Bloom Condition Analysis

# Define bloom threshold (top 25%)
threshold = cyano_species['density_cells_ml'].quantile(0.75)

bloom_events = cyano_species[
    cyano_species['density_cells_ml'] >= threshold
]

bloom_species = (
    bloom_events
    .groupby('sci_name')['density_cells_ml']
    .count()
    .sort_values(ascending=False)
)

bloom_species.head(10)


# 📊 Baseline ML Models
Random Forest & SVR for Cyanobacteria Bloom Prediction

In [ ]:
## CELL 1 — Imports & Setup
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler


In [ ]:
## CELL 2 — Define Target & Features
TARGET = 'density_cells_ml'

EXCLUDE_COLS = [
    TARGET,
    'reservoir'
]

FEATURES = [c for c in df.columns if c not in EXCLUDE_COLS]


In [ ]:
## CELL 3 — Metric Function

def regression_metrics(y_true, y_pred):
    return {
        'R2': r2_score(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred)
    }


In [ ]:
## CELL 4 — Reservoir-wise Train/Test Split (Time-Based)

def time_based_split(sub_df, test_ratio=0.2):
    sub_df = sub_df.sort_values(['year','month'])
    split_idx = int(len(sub_df) * (1 - test_ratio))
    train = sub_df.iloc[:split_idx]
    test = sub_df.iloc[split_idx:]
    return train, test


### 🔹 MODEL 1: RANDOM FOREST REGRESSOR

In [ ]:
## CELL 5 — Train & Evaluate Random Forest (Per Reservoir)

rf_results = []

for reservoir in df['reservoir'].unique():
    sub = df[df['reservoir'] == reservoir]

    if len(sub) < 30:
        continue

    train, test = time_based_split(sub)

    X_train = train[FEATURES]
    y_train = train[TARGET]
    X_test  = test[FEATURES]
    y_test  = test[TARGET]

    rf = RandomForestRegressor(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    )

    rf.fit(X_train, y_train)
    preds = rf.predict(X_test)

    metrics = regression_metrics(y_test, preds)
    metrics['reservoir'] = reservoir
    metrics['model'] = 'RandomForest'

    rf_results.append(metrics)

rf_results_df = pd.DataFrame(rf_results)
rf_results_df.head()


### 🔹 MODEL 2: SUPPORT VECTOR REGRESSION (SVR)

In [ ]:
## CELL 6 — Train & Evaluate SVR (Per Reservoir)

svr_results = []

for reservoir in df['reservoir'].unique():
    sub = df[df['reservoir'] == reservoir]

    if len(sub) < 30:
        continue

    train, test = time_based_split(sub)

    X_train = train[FEATURES]
    y_train = train[TARGET]
    X_test  = test[FEATURES]
    y_test  = test[TARGET]

    # Scaling is REQUIRED for SVR
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)

    svr = SVR(
        kernel='rbf',
        C=10,
        epsilon=0.1
    )

    svr.fit(X_train_scaled, y_train)
    preds = svr.predict(X_test_scaled)

    metrics = regression_metrics(y_test, preds)
    metrics['reservoir'] = reservoir
    metrics['model'] = 'SVR'

    svr_results.append(metrics)

svr_results_df = pd.DataFrame(svr_results)
svr_results_df.head()


In [ ]:
## CELL 7 — Combine Results 

baseline_results = pd.concat([rf_results_df, svr_results_df])
baseline_results


In [ ]:
## CELL 8 — Average Performance Across Reservoirs

baseline_results.groupby('model')[['R2','RMSE','MAE']].mean()


In [ ]:
## CELL 9 — Reservoir-wise Performance Table

baseline_results.pivot_table(
    index='reservoir',
    columns='model',
    values=['R2','RMSE','MAE']
)


# 🚀 XGBoost Reservoir-Wise Modeling

In [ ]:
# CELL 1 - Imports
import numpy as np
import pandas as pd

from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


In [ ]:
# CELL 2 — Define Target & Features
TARGET = 'density_cells_ml'

EXCLUDE_COLS = [
    TARGET,
    'reservoir'
]

FEATURES = [c for c in df.columns if c not in EXCLUDE_COLS]


In [ ]:
# CELL 3 — Metric Function

def regression_metrics(y_true, y_pred):
    return {
        'R2': r2_score(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred)
    }


In [ ]:
# CELL 4 — Time-Based Train/Test Split
def time_based_split(sub_df, test_ratio=0.2):
    sub_df = sub_df.sort_values(['year','month'])
    split_idx = int(len(sub_df) * (1 - test_ratio))
    train = sub_df.iloc[:split_idx]
    test = sub_df.iloc[split_idx:]
    return train, test


In [ ]:
# CELL 5 — XGBoost Reservoir-Wise Training

xgb_results = []

for reservoir in df['reservoir'].unique():
    sub = df[df['reservoir'] == reservoir]

    if len(sub) < 30:
        continue

    train, test = time_based_split(sub)

    X_train = train[FEATURES]
    y_train = train[TARGET]
    X_test  = test[FEATURES]
    y_test  = test[TARGET]

    xgb = XGBRegressor(
        n_estimators=500,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:squarederror',
        random_state=42
    )

    xgb.fit(X_train, y_train)
    preds = xgb.predict(X_test)

    metrics = regression_metrics(y_test, preds)
    metrics['reservoir'] = reservoir
    metrics['model'] = 'XGBoost'

    xgb_results.append(metrics)

xgb_results_df = pd.DataFrame(xgb_results)
xgb_results_df.head()


In [ ]:
# CELL 6 — Reservoir-Wise Performance Table

xgb_results_df.sort_values('R2', ascending=False)


In [ ]:
# CELL 7 — Compare with Baseline Models

all_results = pd.concat([baseline_results, xgb_results_df])

comparison = all_results.pivot_table(
    index='reservoir',
    columns='model',
    values=['R2','RMSE','MAE']
)

comparison


In [ ]:
# CELL 8 — Average Performance Across Reservoirs
all_results.groupby('model')[['R2','RMSE','MAE']].mean()


# 🧠 Hybrid CNN–LSTM Model

In [ ]:
# CELL 1 — Imports

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# CELL 2 — Define Target & Features

TARGET = 'density_cells_ml'

EXCLUDE_COLS = [
    TARGET,
    'reservoir'
]

FEATURES = [c for c in df.columns if c not in EXCLUDE_COLS]


In [ ]:
# CELL 3 — Sequence Builder Function

def create_sequences(X, y, window_size=6):
    X_seq, y_seq = [], []
    for i in range(len(X) - window_size):
        X_seq.append(X[i:i+window_size])
        y_seq.append(y[i+window_size])
    return np.array(X_seq), np.array(y_seq)


In [ ]:
# CELL 4 — Train/Test Split (Time-Based)

def time_series_split(X, y, test_ratio=0.2):
    split = int(len(X) * (1 - test_ratio))
    return X[:split], X[split:], y[:split], y[split:]


### 🔹 CNN–LSTM MODEL DEFINITION

In [ ]:
# CELL 5 — CNN–LSTM Architecture

def build_cnn_lstm(input_shape):
    model = Sequential([
        Conv1D(
            filters=64,
            kernel_size=3,
            activation='relu',
            input_shape=input_shape
        ),
        Dropout(0.3),

        LSTM(64, return_sequences=False),
        Dropout(0.3),

        Dense(32, activation='relu'),
        Dense(1)
    ])

    model.compile(
        optimizer='adam',
        loss='mse'
    )
    return model


In [ ]:
# CELL 6 — Reservoir-wise Training & Evaluation

results = []

WINDOW_SIZE = 6

for reservoir in df['reservoir'].unique():
    sub = df[df['reservoir'] == reservoir].sort_values(['year','month'])

    if len(sub) < 40:
        continue

    X = sub[FEATURES].values
    y = sub[TARGET].values

    # Log-transform target
    y_log = np.log1p(y)

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Create sequences
    X_seq, y_seq = create_sequences(X_scaled, y_log, WINDOW_SIZE)

    # Train-test split
    X_train, X_test, y_train, y_test = time_series_split(X_seq, y_seq)

    model = build_cnn_lstm(
        input_shape=(X_train.shape[1], X_train.shape[2])
    )

    es = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=16,
        validation_split=0.2,
        callbacks=[es],
        verbose=0
    )

    # Predict
    preds_log = model.predict(X_test).flatten()

    # Inverse log transform
    y_test_orig = np.expm1(y_test)
    preds_orig = np.expm1(preds_log)

    metrics = {
        'reservoir': reservoir,
        'model': 'CNN-LSTM',
        'R2': r2_score(y_test_orig, preds_orig),
        'RMSE': np.sqrt(mean_squared_error(y_test_orig, preds_orig)),
        'MAE': mean_absolute_error(y_test_orig, preds_orig)
    }

    results.append(metrics)

cnn_lstm_results = pd.DataFrame(results)
cnn_lstm_results


In [ ]:
# CELL 7 — Compare with Previous Models

final_results = pd.concat([all_results, cnn_lstm_results])

final_results.groupby('model')[['R2','RMSE','MAE']].mean()


# 🔷 Transformer Time-Series Model

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Dense, Dropout, LayerNormalization,
    MultiHeadAttention, Add, Embedding,
    GlobalAveragePooling1D, Flatten, Concatenate
)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import Huber


In [ ]:
# CELL 2 — Target & Feature Selection
TARGET = 'density_cells_ml'

# Seasonal encoding
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

EXCLUDE_COLS = [
    TARGET,
    'reservoir',
    'month'
]

FEATURES = [c for c in df.columns if c not in EXCLUDE_COLS]


In [ ]:
# CELL 3 — Reservoir Encoding

df['reservoir_id'] = df['reservoir'].astype('category').cat.codes
FEATURES.append('reservoir_id')

NUM_RESERVOIRS = df['reservoir_id'].nunique()


In [ ]:
# CELL 4 — Multi-Horizon Sequence Generator (CRITICAL)

def create_multi_horizon_sequences(df, features, target, window=12, horizon=6):
    X, y, res_ids = [], [], []

    for res in df['reservoir'].unique():
        sub = df[df['reservoir'] == res].sort_values(['year','month'])

        if len(sub) <= window + horizon:
            continue

        X_res = sub[features].values
        y_res = np.log1p(sub[target].values)
        rid   = sub['reservoir_id'].values

        for i in range(len(sub) - window - horizon + 1):
            X.append(X_res[i:i+window])
            y.append(y_res[i+window:i+window+horizon])
            res_ids.append(rid[i+window])

    return np.array(X), np.array(y), np.array(res_ids)


In [ ]:
# CELL 5 — Transformer Encoder (Attention Extractable)

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.1):

    attn_layer = MultiHeadAttention(
        key_dim=head_size,
        num_heads=num_heads,
        dropout=dropout
    )

    attn_output = attn_layer(inputs, inputs)

    x = Dropout(dropout)(attn_output)
    x = Add()([x, inputs])
    x = LayerNormalization(epsilon=1e-6)(x)

    ff = Dense(ff_dim, activation='relu')(x)
    ff = Dense(inputs.shape[-1])(ff)
    x = Add()([ff, x])

    return LayerNormalization(epsilon=1e-6)(x), attn_layer


In [ ]:
# CELL 6 — FULL TRANSFORMER MODEL (Multi-Horizon + Embedding)

def build_transformer(input_shape, num_reservoirs, horizon=6, embed_dim=8):

    ts_input = Input(shape=input_shape, name='ts_input')
    res_input = Input(shape=(1,), name='res_input')

    # Reservoir embedding
    res_embed = Embedding(
        input_dim=num_reservoirs,
        output_dim=embed_dim
    )(res_input)
    res_embed = Flatten()(res_embed)

    # Transformer block
    x, attn_layer = transformer_encoder(
        ts_input,
        head_size=64,
        num_heads=4,
        ff_dim=128,
        dropout=0.2
    )

    x = GlobalAveragePooling1D()(x)

    x = Concatenate()([x, res_embed])

    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)

    outputs = Dense(horizon)(x)

    model = Model(
        inputs=[ts_input, res_input],
        outputs=outputs
    )

    model.compile(
        optimizer='adam',
        loss=Huber(delta=1.0)
    )

    # Store attention layer for later extraction
    model.attn_layer = attn_layer

    return model


In [ ]:
# CELL 7 — Create Sequences & Scale

WINDOW = 12
HORIZON = 6

X_seq, y_seq, res_id_seq = create_multi_horizon_sequences(
    df,
    FEATURES,
    TARGET,
    window=WINDOW,
    horizon=HORIZON
)

scaler = StandardScaler()
n, t, f = X_seq.shape

X_scaled = scaler.fit_transform(
    X_seq.reshape(-1, f)
).reshape(n, t, f)


In [ ]:
# CELL 8 — Time-Based Split

split = int(len(X_scaled) * 0.8)

X_train, X_test = X_scaled[:split], X_scaled[split:]
y_train, y_test = y_seq[:split], y_seq[split:]
res_id_train, res_id_test = res_id_seq[:split], res_id_seq[split:]


In [ ]:
# CELL 9 — Train Model

model = build_transformer(
    input_shape=(X_train.shape[1], X_train.shape[2]),
    num_reservoirs=NUM_RESERVOIRS,
    horizon=HORIZON
)

es = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

model.fit(
    [X_train, res_id_train],
    y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[es],
    verbose=1
)


In [ ]:
# CELL 10 — Multi-Horizon Evaluation

preds_log = model.predict([X_test, res_id_test])

preds_orig = np.expm1(preds_log)
y_test_orig = np.expm1(y_test)

for h in range(HORIZON):
    r2 = r2_score(y_test_orig[:,h], preds_orig[:,h])
    rmse = np.sqrt(mean_squared_error(y_test_orig[:,h], preds_orig[:,h]))
    mae = mean_absolute_error(y_test_orig[:,h], preds_orig[:,h])

    print(f"Month +{h+1}: R2={r2:.3f}, RMSE={rmse:.1f}, MAE={mae:.1f}")


In [ ]:
# CELL 11 — Attention Weight Visualization (CORRECT)

# Build attention probe model
attn_probe = Model(
    inputs=model.inputs,
    outputs=model.attn_layer(
        model.inputs[0],
        model.inputs[0],
        return_attention_scores=True
    )[1]
)

# Extract attention for one sample
attn_scores = attn_probe.predict([X_test[:1], res_id_test[:1]])

# Average across heads and feature dimensions
avg_attn = attn_scores.mean(axis=(1, 2))


In [ ]:
# CELL 12 — Plot Temporal Attention

import matplotlib.pyplot as plt

plt.figure(figsize=(10,4))
plt.plot(avg_attn[0])
plt.xlabel("Past Months (Time Steps)")
plt.ylabel("Attention Weight")
plt.title("Transformer Temporal Attention Pattern")
plt.grid(True)
plt.show()


# 🧠 Graph Neural Network (GNN)

In [ ]:
!pip install torch-geometric

In [ ]:
# CELL 1 - Imports

import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
# CELL 2 — Prepare Node Features (Reservoir-Level) 

feature_cols = [
    c for c in df.columns
    if c not in ['density_cells_ml', 'reservoir']
]

node_features = (
    df
    .groupby('reservoir')[feature_cols]
    .mean()
)

node_features_scaled = (
    node_features - node_features.mean()
) / node_features.std()

X_nodes = torch.tensor(
    node_features_scaled.values,
    dtype=torch.float
)


In [ ]:
# CELL 3 — Target Variable (Node-Level)
# We predict mean bloom intensity per reservoir.
y_nodes = (
    df
    .groupby('reservoir')['density_cells_ml']
    .mean()
)

y_nodes = torch.tensor(
    y_nodes.values,
    dtype=torch.float
)


In [ ]:
# CELL 4 — Build Graph Edges via Similarity

similarity = cosine_similarity(node_features_scaled.values)

edge_index = []

K = 3  # each reservoir connects to its 3 most similar reservoirs

for i in range(similarity.shape[0]):
    # Get indices of top-K similar reservoirs (excluding itself)
    neighbors = np.argsort(similarity[i])[-(K+1):-1]

    for j in neighbors:
        edge_index.append([i, j])
        edge_index.append([j, i])  # make graph undirected

edge_index = torch.tensor(edge_index, dtype=torch.long).t()

print("Edge index shape:", edge_index.shape)


In [ ]:
# CELL 5 — Create PyG Graph Object 
data = Data(
    x=X_nodes,
    edge_index=edge_index,
    y=y_nodes
)


In [ ]:
# CELL 6 — Define GNN Model (GCN) 

class ReservoirGNN(torch.nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, 32)
        self.lin = torch.nn.Linear(32, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.conv2(x, edge_index)
        x = F.relu(x)

        return self.lin(x).squeeze()


In [ ]:
# CELL 7 — Train/Test Split (Node-Level)

num_nodes = data.num_nodes
indices = np.arange(num_nodes)

np.random.seed(42)
np.random.shuffle(indices)

split = int(0.8 * num_nodes)
train_idx = indices[:split]
test_idx = indices[split:]

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[train_idx] = True
test_mask[test_idx] = True

data.train_mask = train_mask
data.test_mask = test_mask


In [ ]:
# CELL 8 — Train GNN

model = ReservoirGNN(in_channels=X_nodes.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(300):
    model.train()
    optimizer.zero_grad()

    out = model(data)
    loss = F.mse_loss(out[data.train_mask], data.y[data.train_mask])

    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


In [ ]:
# CELL 9 — Evaluate GNN

model.eval()
with torch.no_grad():
    preds = model(data)

y_true = data.y[data.test_mask].numpy()
y_pred = preds[data.test_mask].numpy()

gnn_metrics = {
    'model': 'GNN',
    'R2': r2_score(y_true, y_pred),
    'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
    'MAE': mean_absolute_error(y_true, y_pred)
}

gnn_metrics


In [ ]:
# CELL 10 — Compare with All Models
final_results = pd.concat([
    final_results,
    pd.DataFrame([gnn_metrics])
])

final_results.groupby('model')[['R2','RMSE','MAE']].mean()